In [1]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.250:7077") \
        .appName("AlmaLundbergSparkapplicationA3")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.cores.max", 4)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 21:23:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/06 21:23:55 INFO StandaloneSchedulerBackend$StandaloneDriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (192.168.2.250:36806) with ID 0,  ResourceProfileId 0
24/03/06 21:23:55 INFO StandaloneSchedulerBackend$StandaloneDriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (192.168.2.250:36792) with ID 1,  ResourceProfileId 0
24/03/06 21:23:55 INFO BlockManagerMasterEndpoint: Registering block manager 192.168.2.250:10006 with 366.3 MiB RAM, BlockManagerId(0, 192.168.2.250, 10006, None)
24/03/06 21:23:55 INFO BlockManagerMasterEndpoint: Registering block manager 192.168.2.250:10005 with 366.3 MiB RAM, BlockManagerId(1, 192.168.2.250, 10005, None)


In [2]:
# Loading the text file from HDFS
lines_en = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.en")
num_lines_en = lines_en.count()
print(f"Total lines in English file: {num_lines_en}") 

lines_sv = spark_context.textFile("hdfs://192.168.2.250:9000/europarl/europarl-v7.sv-en.sv")
num_lines_sv = lines_sv.count()
print(f"Total lines in Swedish file: {num_lines_sv}")

24/03/06 16:40:50 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 221.5 KiB, free 434.2 MiB)
24/03/06 16:40:50 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 32.6 KiB, free 434.2 MiB)
24/03/06 16:40:50 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on host-192-168-2-147-de1:10005 (size: 32.6 KiB, free: 434.4 MiB)
24/03/06 16:40:50 INFO SparkContext: Created broadcast 0 from textFile at NativeMethodAccessorImpl.java:0
24/03/06 16:40:51 INFO FileInputFormat: Total input files to process : 1
24/03/06 16:40:51 INFO NetworkTopology: Adding a new node: /default-rack/192.168.2.250:9866
24/03/06 16:40:51 INFO SparkContext: Starting job: count at /tmp/ipykernel_14848/3631255496.py:4
24/03/06 16:40:51 INFO DAGScheduler: Got job 0 (count at /tmp/ipykernel_14848/3631255496.py:4) with 2 output partitions
24/03/06 16:40:51 INFO DAGScheduler: Final stage: ResultStage 0 (count at /tmp/ipykernel_14848/3631255496.py:4)
24/03

Total lines in English file: 1862234


24/03/06 16:40:58 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on 192.168.2.250:10005 (size: 5.5 KiB, free: 366.3 MiB)
24/03/06 16:40:58 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on 192.168.2.250:10006 (size: 5.5 KiB, free: 366.3 MiB)
24/03/06 16:40:58 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 192.168.2.250:10005 (size: 32.6 KiB, free: 366.2 MiB)
24/03/06 16:40:58 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on 192.168.2.250:10006 (size: 32.6 KiB, free: 366.2 MiB)
24/03/06 16:40:59 INFO TaskSetManager: Finished task 2.0 in stage 1.0 (TID 4) in 327 ms on 192.168.2.250 (executor 1) (1/3)


Total lines in Swedish file: 1862234


24/03/06 16:41:00 INFO TaskSetManager: Finished task 1.0 in stage 1.0 (TID 3) in 1466 ms on 192.168.2.250 (executor 0) (2/3)
24/03/06 16:41:00 INFO TaskSetManager: Finished task 0.0 in stage 1.0 (TID 2) in 1614 ms on 192.168.2.250 (executor 1) (3/3)
24/03/06 16:41:00 INFO TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
24/03/06 16:41:00 INFO DAGScheduler: ResultStage 1 (count at /tmp/ipykernel_14848/3631255496.py:8) finished in 1.647 s
24/03/06 16:41:00 INFO DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/06 16:41:00 INFO TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
24/03/06 16:41:00 INFO DAGScheduler: Job 1 finished: count at /tmp/ipykernel_14848/3631255496.py:8, took 1.655664 s


In [3]:
# Counting nmr of partitions in the English file
num_partitions_en = lines_en.getNumPartitions()
print(f"Number of partitions in English file: {num_partitions_en}")

# Counting nmr of partitions in the Swedish file
num_partitions_sv = lines_sv.getNumPartitions()
print(f"Number of partitions in Swedish file: {num_partitions_sv}")

Number of partitions in English file: 2
Number of partitions in Swedish file: 3


In [6]:
def preprocess_text(line):
    return line.lower().split(' ')

In [7]:
processed_lines_en = lines_en.map(preprocess_text)
processed_lines_sv = lines_sv.map(preprocess_text)

In [8]:
# Verify that the line counts still match after the pre-processing:

processed_count_en = processed_lines_en.count()
processed_count_sv = processed_lines_sv.count()
print(f"Processed line count in English: {processed_count_en}")
print(f"Processed line count in Swedish: {processed_count_sv}")

24/03/06 16:41:36 INFO SparkContext: Starting job: count at /tmp/ipykernel_14848/226898882.py:3
24/03/06 16:41:36 INFO DAGScheduler: Got job 2 (count at /tmp/ipykernel_14848/226898882.py:3) with 2 output partitions
24/03/06 16:41:36 INFO DAGScheduler: Final stage: ResultStage 2 (count at /tmp/ipykernel_14848/226898882.py:3)
24/03/06 16:41:36 INFO DAGScheduler: Parents of final stage: List()
24/03/06 16:41:36 INFO DAGScheduler: Missing parents: List()
24/03/06 16:41:36 INFO DAGScheduler: Submitting ResultStage 2 (PythonRDD[6] at count at /tmp/ipykernel_14848/226898882.py:3), which has no missing parents
24/03/06 16:41:36 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 9.6 KiB, free 433.9 MiB)
24/03/06 16:41:36 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 5.7 KiB, free 433.9 MiB)
24/03/06 16:41:36 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on host-192-168-2-147-de1:10005 (size: 5.7 KiB, free: 434.3 MiB)

Processed line count in English: 1862234
Processed line count in Swedish: 1862234


24/03/06 16:41:45 INFO TaskSetManager: Finished task 1.0 in stage 3.0 (TID 8) in 3058 ms on 192.168.2.250 (executor 1) (2/3)
24/03/06 16:41:45 INFO TaskSetManager: Finished task 0.0 in stage 3.0 (TID 7) in 3113 ms on 192.168.2.250 (executor 0) (3/3)
24/03/06 16:41:45 INFO TaskSchedulerImpl: Removed TaskSet 3.0, whose tasks have all completed, from pool 
24/03/06 16:41:45 INFO DAGScheduler: ResultStage 3 (count at /tmp/ipykernel_14848/226898882.py:4) finished in 3.133 s
24/03/06 16:41:45 INFO DAGScheduler: Job 3 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/06 16:41:45 INFO TaskSchedulerImpl: Killing all running tasks in stage 3: Stage finished
24/03/06 16:41:45 INFO DAGScheduler: Job 3 finished: count at /tmp/ipykernel_14848/226898882.py:4, took 3.137989 s


In [82]:
# The line count still match after pre-processing

In [9]:
# Check 10 most frequent words in English and Swedish

words_en = processed_lines_en.flatMap(lambda line: line)
words_sv = processed_lines_sv.flatMap(lambda line: line)

word_counts_en = words_en.map(lambda word: (word, 1)).reduceByKey(add)
word_counts_sv = words_sv.map(lambda word: (word, 1)).reduceByKey(add)

top_10_words_en = word_counts_en.takeOrdered(10, key=lambda x: -x[1])
top_10_words_sv = word_counts_sv.takeOrdered(10, key=lambda x: -x[1])

print("\nTop 10 words in English:")
for word, count in top_10_words_en:
    print(f'{word}: {count}')
    
print("\nTop 10 words in Swedish:")
for word, count in top_10_words_sv:
    print(f'{word}: {count}')

24/03/06 16:41:48 INFO SparkContext: Starting job: takeOrdered at /tmp/ipykernel_14848/2132416953.py:9
24/03/06 16:41:48 INFO DAGScheduler: Registering RDD 9 (reduceByKey at /tmp/ipykernel_14848/2132416953.py:6) as input to shuffle 0
24/03/06 16:41:48 INFO DAGScheduler: Got job 4 (takeOrdered at /tmp/ipykernel_14848/2132416953.py:9) with 2 output partitions
24/03/06 16:41:48 INFO DAGScheduler: Final stage: ResultStage 5 (takeOrdered at /tmp/ipykernel_14848/2132416953.py:9)
24/03/06 16:41:48 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 4)
24/03/06 16:41:48 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 4)
24/03/06 16:41:48 INFO DAGScheduler: Submitting ShuffleMapStage 4 (PairwiseRDD[9] at reduceByKey at /tmp/ipykernel_14848/2132416953.py:6), which has no missing parents
24/03/06 16:41:48 INFO MemoryStore: Block broadcast_6 stored as values in memory (estimated size 12.9 KiB, free 433.9 MiB)
24/03/06 16:41:48 INFO MemoryStore: Block broadcast_6_piece0 stored 


Top 10 words in English:
the: 3498375
of: 1659758
to: 1539760
and: 1288401
in: 1085993
that: 797516
a: 773522
is: 758050
for: 534242
we: 522849

Top 10 words in Swedish:
att: 1706293
och: 1344830
i: 1050774
det: 924866
som: 913276
för: 908680
av: 738068
är: 694381
en: 620310
vi: 539797


24/03/06 16:42:22 INFO TaskSetManager: Finished task 2.0 in stage 7.0 (TID 19) in 294 ms on 192.168.2.250 (executor 1) (1/3)
24/03/06 16:42:22 INFO TaskSetManager: Finished task 1.0 in stage 7.0 (TID 18) in 312 ms on 192.168.2.250 (executor 0) (2/3)
24/03/06 16:42:22 INFO TaskSetManager: Finished task 0.0 in stage 7.0 (TID 17) in 334 ms on 192.168.2.250 (executor 1) (3/3)
24/03/06 16:42:22 INFO DAGScheduler: ResultStage 7 (takeOrdered at /tmp/ipykernel_14848/2132416953.py:10) finished in 0.350 s
24/03/06 16:42:22 INFO TaskSchedulerImpl: Removed TaskSet 7.0, whose tasks have all completed, from pool 
24/03/06 16:42:22 INFO DAGScheduler: Job 5 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/06 16:42:22 INFO TaskSchedulerImpl: Killing all running tasks in stage 7: Stage finished
24/03/06 16:42:22 INFO DAGScheduler: Job 5 finished: takeOrdered at /tmp/ipykernel_14848/2132416953.py:10, took 15.219807 s


In [84]:
# The results are reasonable, as the most frequent words are common words used to build up sentences (konjunctions, prepositions etc)
# In the Swedish translation for example, the words also match most of the top 10 most frequent words in the Swedish language like i, och, att, det, som, en, är, av, för 

In [16]:
# Question A4

# Assign a unique index to each line in both RDDs and swap so that the line number is the key
sv_swapped = processed_lines_sv.zipWithIndex().map(lambda x: (x[1], x[0]))
en_swapped = processed_lines_en.zipWithIndex().map(lambda x: (x[1], x[0]))

24/03/06 16:43:59 INFO SparkContext: Starting job: zipWithIndex at /tmp/ipykernel_14848/765333879.py:4
24/03/06 16:43:59 INFO DAGScheduler: Got job 11 (zipWithIndex at /tmp/ipykernel_14848/765333879.py:4) with 3 output partitions
24/03/06 16:43:59 INFO DAGScheduler: Final stage: ResultStage 13 (zipWithIndex at /tmp/ipykernel_14848/765333879.py:4)
24/03/06 16:43:59 INFO DAGScheduler: Parents of final stage: List()
24/03/06 16:43:59 INFO DAGScheduler: Missing parents: List()
24/03/06 16:43:59 INFO DAGScheduler: Submitting ResultStage 13 (PythonRDD[23] at zipWithIndex at /tmp/ipykernel_14848/765333879.py:4), which has no missing parents
24/03/06 16:43:59 INFO MemoryStore: Block broadcast_15 stored as values in memory (estimated size 8.3 KiB, free 433.9 MiB)
24/03/06 16:43:59 INFO MemoryStore: Block broadcast_15_piece0 stored as bytes in memory (estimated size 5.1 KiB, free 433.9 MiB)
24/03/06 16:43:59 INFO BlockManagerInfo: Added broadcast_15_piece0 in memory on host-192-168-2-147-de1:100

In [17]:
# See how it looks, so everything was done correctly
sv_swapped.take(10)

24/03/06 16:44:45 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/06 16:44:45 INFO DAGScheduler: Got job 13 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/06 16:44:45 INFO DAGScheduler: Final stage: ResultStage 15 (runJob at PythonRDD.scala:181)
24/03/06 16:44:45 INFO DAGScheduler: Parents of final stage: List()
24/03/06 16:44:45 INFO DAGScheduler: Missing parents: List()
24/03/06 16:44:45 INFO DAGScheduler: Submitting ResultStage 15 (PythonRDD[25] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/06 16:44:45 INFO MemoryStore: Block broadcast_17 stored as values in memory (estimated size 9.2 KiB, free 433.9 MiB)
24/03/06 16:44:45 INFO MemoryStore: Block broadcast_17_piece0 stored as bytes in memory (estimated size 5.6 KiB, free 433.9 MiB)
24/03/06 16:44:45 INFO BlockManagerInfo: Added broadcast_17_piece0 in memory on host-192-168-2-147-de1:10005 (size: 5.6 KiB, free: 434.3 MiB)
24/03/06 16:44:45 INFO SparkContext: Created broadcast 1

[(0, ['återupptagande', 'av', 'sessionen']),
 (1,
  ['jag',
   'förklarar',
   'europaparlamentets',
   'session',
   'återupptagen',
   'efter',
   'avbrottet',
   'den',
   '17',
   'december.',
   'jag',
   'vill',
   'på',
   'nytt',
   'önska',
   'er',
   'ett',
   'gott',
   'nytt',
   'år',
   'och',
   'jag',
   'hoppas',
   'att',
   'ni',
   'haft',
   'en',
   'trevlig',
   'semester.']),
 (2,
  ['som',
   'ni',
   'kunnat',
   'konstatera',
   'ägde',
   '"den',
   'stora',
   'år',
   '2000-buggen"',
   'aldrig',
   'rum.',
   'däremot',
   'har',
   'invånarna',
   'i',
   'ett',
   'antal',
   'av',
   'våra',
   'medlemsländer',
   'drabbats',
   'av',
   'naturkatastrofer',
   'som',
   'verkligen',
   'varit',
   'förskräckliga.']),
 (3,
  ['ni',
   'har',
   'begärt',
   'en',
   'debatt',
   'i',
   'ämnet',
   'under',
   'sammanträdesperiodens',
   'kommande',
   'dagar.']),
 (4,
  ['till',
   'dess',
   'vill',
   'jag',
   'att',
   'vi,',
   'som',
   'ett',
 

In [18]:
# See how it looks, so everything was done correctly
en_swapped.take(10)

24/03/06 16:44:56 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/06 16:44:56 INFO DAGScheduler: Got job 14 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/06 16:44:56 INFO DAGScheduler: Final stage: ResultStage 16 (runJob at PythonRDD.scala:181)
24/03/06 16:44:56 INFO DAGScheduler: Parents of final stage: List()
24/03/06 16:44:56 INFO DAGScheduler: Missing parents: List()
24/03/06 16:44:56 INFO DAGScheduler: Submitting ResultStage 16 (PythonRDD[26] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/06 16:44:56 INFO MemoryStore: Block broadcast_18 stored as values in memory (estimated size 9.2 KiB, free 433.9 MiB)
24/03/06 16:44:56 INFO MemoryStore: Block broadcast_18_piece0 stored as bytes in memory (estimated size 5.6 KiB, free 433.9 MiB)
24/03/06 16:44:56 INFO BlockManagerInfo: Added broadcast_18_piece0 in memory on host-192-168-2-147-de1:10005 (size: 5.6 KiB, free: 434.3 MiB)
24/03/06 16:44:56 INFO SparkContext: Created broadcast 1

[(0, ['resumption', 'of', 'the', 'session']),
 (1,
  ['i',
   'declare',
   'resumed',
   'the',
   'session',
   'of',
   'the',
   'european',
   'parliament',
   'adjourned',
   'on',
   'friday',
   '17',
   'december',
   '1999,',
   'and',
   'i',
   'would',
   'like',
   'once',
   'again',
   'to',
   'wish',
   'you',
   'a',
   'happy',
   'new',
   'year',
   'in',
   'the',
   'hope',
   'that',
   'you',
   'enjoyed',
   'a',
   'pleasant',
   'festive',
   'period.']),
 (2,
  ['although,',
   'as',
   'you',
   'will',
   'have',
   'seen,',
   'the',
   'dreaded',
   "'millennium",
   "bug'",
   'failed',
   'to',
   'materialise,',
   'still',
   'the',
   'people',
   'in',
   'a',
   'number',
   'of',
   'countries',
   'suffered',
   'a',
   'series',
   'of',
   'natural',
   'disasters',
   'that',
   'truly',
   'were',
   'dreadful.']),
 (3,
  ['you',
   'have',
   'requested',
   'a',
   'debate',
   'on',
   'this',
   'subject',
   'in',
   'the',
   'course

In [19]:
# Join the two RDDs together on the line number key
sv_en_joined_rdd = sv_swapped.join(en_swapped)

In [20]:
# See how it looks
sv_en_joined_rdd.take(10) 

24/03/06 16:45:01 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/06 16:45:01 INFO DAGScheduler: Registering RDD 31 (join at /tmp/ipykernel_14848/3095717598.py:2) as input to shuffle 2
24/03/06 16:45:01 INFO DAGScheduler: Got job 15 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/06 16:45:01 INFO DAGScheduler: Final stage: ResultStage 18 (runJob at PythonRDD.scala:181)
24/03/06 16:45:01 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 17)
24/03/06 16:45:01 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 17)
24/03/06 16:45:01 INFO DAGScheduler: Submitting ShuffleMapStage 17 (PairwiseRDD[31] at join at /tmp/ipykernel_14848/3095717598.py:2), which has no missing parents
24/03/06 16:45:01 INFO MemoryStore: Block broadcast_19 stored as values in memory (estimated size 18.5 KiB, free 433.9 MiB)
24/03/06 16:45:01 INFO MemoryStore: Block broadcast_19_piece0 stored as bytes in memory (estimated size 8.7 KiB, free 433.9 MiB)
24/03/06 16

[(6305,
  (['den',
    'största',
    'svagheten',
    'är',
    'som',
    'jag',
    'ser',
    'det',
    'att',
    'man',
    'inte',
    'har',
    'tagit',
    'tillräckligt',
    'stor',
    'hänsyn',
    'till',
    'de',
    'medelstora',
    'företagens',
    'situation,',
    'men',
    'bilindustrin',
    'består',
    'inte',
    'endast',
    'av',
    'storföretag.'],
   ['there',
    'are',
    'certain',
    'weaknesses,',
    'the',
    'greatest',
    'of',
    'which,',
    'in',
    'my',
    'opinion,',
    'is',
    'that',
    'not',
    'enough',
    'consideration',
    'is',
    'being',
    'given',
    'to',
    'the',
    'situation',
    'of',
    'small',
    'businesses;',
    'after',
    'all,',
    'the',
    'car',
    'industry',
    'is',
    'more',
    'than',
    'just',
    'large',
    'companies.'])),
 (6845,
  (['europaparlamentet',
    'har',
    'aldrig',
    'tvekat',
    'att',
    'kommentera',
    'en',
    'utveckling',
    'man',
 

In [21]:
# Filter function for the joined RDD according to lab requirements
def filter(lines):
    #Filter to exclude empty/missing lines
    filtered_rdd = lines.filter(lambda x: len(x[1][0]) > 1 and len(x[1][1]) > 1)

    #Filter sentences with a small number (the threshold) of words
    threshold = 5
    filtered_rdd = filtered_rdd.filter(lambda x: len(x[1][0]) < threshold and len(x[1][1]) < threshold)

    # Filter pairs with the same number of words
    filtered_rdd = filtered_rdd.filter(lambda x: len(x[1][0]) == len(x[1][1]))
    return filtered_rdd

In [22]:
# Call on the filter function to filter the joined RDD
filtered_rdd = filter(sv_en_joined_rdd)

In [15]:
# See how it looks, if it was filtered properly
filtered_rdd.take(10)

24/03/06 16:15:39 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/06 16:15:39 INFO DAGScheduler: Got job 11 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/06 16:15:39 INFO DAGScheduler: Final stage: ResultStage 15 (runJob at PythonRDD.scala:181)
24/03/06 16:15:39 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 14)
24/03/06 16:15:39 INFO DAGScheduler: Missing parents: List()
24/03/06 16:15:39 INFO DAGScheduler: Submitting ResultStage 15 (PythonRDD[30] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/06 16:15:39 INFO MemoryStore: Block broadcast_16 stored as values in memory (estimated size 14.2 KiB, free 433.8 MiB)
24/03/06 16:15:39 INFO MemoryStore: Block broadcast_16_piece0 stored as bytes in memory (estimated size 7.7 KiB, free 433.9 MiB)
24/03/06 16:15:39 INFO BlockManagerInfo: Added broadcast_16_piece0 in memory on host-192-168-2-147-de1:10005 (size: 7.7 KiB, free: 434.3 MiB)
24/03/06 16:15:39 INFO SparkContext: 

[(916685, (['det', 'är', 'inte', 'rätt.'], ['that', 'is', 'not', 'right.'])),
 (950055, (['(tumult', 'och', 'applåder)'], ['(uproar', 'and', 'applause)'])),
 (1030115, (['inga', 'ändringar.'], ['no', 'amendments'])),
 (1093240,
  (['skriftliga', 'förklaringar', '(artikel', '142)'],
   ['written', 'statements', '(rule', '142)'])),
 (1114365, (['i', 'vilket', 'forum?'], ['in', 'which', 'forum?'])),
 (1118050, (['detta', 'måste', 'ändras.'], ['this', 'must', 'change.'])),
 (1491205,
  (['debatten', 'är', 'härmed', 'avslutad.'],
   ['the', 'debate', 'is', 'closed.'])),
 (1426720,
  (['debatten', 'är', 'härmed', 'avslutad.'],
   ['the', 'debate', 'is', 'closed.'])),
 (1581900,
  (['finns', 'det', 'några', 'synpunkter?'],
   ['are', 'there', 'any', 'comments?'])),
 (993145, (['varför', 'inte?'], ['why', 'not?']))]

In [23]:
# Pair words in the order they appear in each sentence pair
word_pairs_rdd = filtered_rdd.map(lambda x: zip(x[1][0], x[1][1]))

In [24]:
# See how it looks
print(list(word_pairs_rdd.take(1)[0])) 

24/03/06 16:46:37 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/06 16:46:37 INFO DAGScheduler: Got job 16 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/06 16:46:37 INFO DAGScheduler: Final stage: ResultStage 20 (runJob at PythonRDD.scala:181)
24/03/06 16:46:37 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 19)
24/03/06 16:46:37 INFO DAGScheduler: Missing parents: List()
24/03/06 16:46:37 INFO DAGScheduler: Submitting ResultStage 20 (PythonRDD[35] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/06 16:46:37 INFO MemoryStore: Block broadcast_21 stored as values in memory (estimated size 14.5 KiB, free 433.9 MiB)
24/03/06 16:46:37 INFO MemoryStore: Block broadcast_21_piece0 stored as bytes in memory (estimated size 7.8 KiB, free 433.9 MiB)
24/03/06 16:46:37 INFO BlockManagerInfo: Added broadcast_21_piece0 in memory on host-192-168-2-147-de1:10005 (size: 7.8 KiB, free: 434.3 MiB)
24/03/06 16:46:37 INFO BlockManagerIn

[('skriftliga', 'written'), ('förklaringar', 'statements'), ('(artikel', '(rule'), ('149)', '149)')]


24/03/06 16:46:56 INFO TaskSetManager: Finished task 0.0 in stage 20.0 (TID 41) in 18598 ms on 192.168.2.250 (executor 1) (1/1)
24/03/06 16:46:56 INFO TaskSchedulerImpl: Removed TaskSet 20.0, whose tasks have all completed, from pool 
24/03/06 16:46:56 INFO DAGScheduler: ResultStage 20 (runJob at PythonRDD.scala:181) finished in 18.623 s
24/03/06 16:46:56 INFO DAGScheduler: Job 16 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/06 16:46:56 INFO TaskSchedulerImpl: Killing all running tasks in stage 20: Stage finished
24/03/06 16:46:56 INFO DAGScheduler: Job 16 finished: runJob at PythonRDD.scala:181, took 18.631341 s


In [25]:
# Count the frequency of each word-translation pair
words_rdd = word_pairs_rdd.flatMap(lambda line: line)
word_counts_rdd = words_rdd.map(lambda word: (word, 1))
word_freq_rdd = word_counts_rdd.reduceByKey(lambda a, b: a + b)

# Print 10 of the most frequently occurring pairs of words
top_pairs = word_freq_rdd.takeOrdered(10, key=lambda x: -x[1])
for (word, translation), count in top_pairs:
    print(f'{word} - {translation}: {count}')


24/03/06 16:47:07 INFO SparkContext: Starting job: takeOrdered at /tmp/ipykernel_14848/2833874270.py:7
24/03/06 16:47:07 INFO DAGScheduler: Registering RDD 37 (reduceByKey at /tmp/ipykernel_14848/2833874270.py:4) as input to shuffle 3
24/03/06 16:47:07 INFO DAGScheduler: Got job 17 (takeOrdered at /tmp/ipykernel_14848/2833874270.py:7) with 5 output partitions
24/03/06 16:47:07 INFO DAGScheduler: Final stage: ResultStage 23 (takeOrdered at /tmp/ipykernel_14848/2833874270.py:7)
24/03/06 16:47:07 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 22)
24/03/06 16:47:07 INFO DAGScheduler: Missing parents: List(ShuffleMapStage 22)
24/03/06 16:47:07 INFO DAGScheduler: Submitting ShuffleMapStage 22 (PairwiseRDD[37] at reduceByKey at /tmp/ipykernel_14848/2833874270.py:4), which has no missing parents
24/03/06 16:47:08 INFO MemoryStore: Block broadcast_22 stored as values in memory (estimated size 17.0 KiB, free 433.9 MiB)
24/03/06 16:47:08 INFO MemoryStore: Block broadcast_22_piece

avslutad. - closed.: 2534
är - is: 2380
jag - the: 1324
debatten - is: 1324
förklarar - debate: 1317
debatten - the: 1225
härmed - is: 1215
är - debate: 1187
(artikel - (rule: 893
det - that: 852


24/03/06 16:47:57 INFO TaskSetManager: Finished task 4.0 in stage 22.0 (TID 46) in 23582 ms on 192.168.2.250 (executor 0) (5/5)
24/03/06 16:47:57 INFO TaskSchedulerImpl: Removed TaskSet 22.0, whose tasks have all completed, from pool 
24/03/06 16:47:57 INFO DAGScheduler: ShuffleMapStage 22 (reduceByKey at /tmp/ipykernel_14848/2833874270.py:4) finished in 49.813 s
24/03/06 16:47:57 INFO DAGScheduler: looking for newly runnable stages
24/03/06 16:47:57 INFO DAGScheduler: running: Set()
24/03/06 16:47:57 INFO DAGScheduler: waiting: Set(ResultStage 23)
24/03/06 16:47:57 INFO DAGScheduler: failed: Set()
24/03/06 16:47:57 INFO DAGScheduler: Submitting ResultStage 23 (PythonRDD[40] at takeOrdered at /tmp/ipykernel_14848/2833874270.py:7), which has no missing parents
24/03/06 16:47:57 INFO MemoryStore: Block broadcast_23 stored as values in memory (estimated size 11.2 KiB, free 433.9 MiB)
24/03/06 16:47:57 INFO MemoryStore: Block broadcast_23_piece0 stored as bytes in memory (estimated size 6.

In [ ]:
# 10 most frequent word pairs: 
# avslutad. - closed.: 2534
# är - is: 2380
# jag - the: 1324
# debatten - is: 1324
# förklarar - debate: 1317
# debatten - the: 1225
# härmed - is: 1215
# är - debate: 1187
# (artikel - (rule: 893
# det - that: 852

# The translations are not parfect, and the word pairs doesnt match in translation. However, the first two makes sens, and the last two. 
# The other ones are not translations of each other, but it looks like one of the most frequent short sentences perhaps could be 
# "härmed förklarar jag debatten avslutad", which is a completely different word structure on english - why the word pairs are a bit "scrambled"
# That is my conclusion at least.

In [2]:
# Section B - Working with DataFrames and SQL

dataframe = spark_session.read.csv("hdfs://192.168.2.250:9000/parking-citations.csv", header = True, inferSchema=True)
dataframe.show()

24/03/06 21:27:01 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/06 21:27:01 INFO SharedState: Warehouse path is 'file:/home/ubuntu/dataengineering_AlmaLundberg/A3/spark-warehouse'.
24/03/06 21:27:02 INFO InMemoryFileIndex: It took 117 ms to list leaf files for 1 paths.
24/03/06 21:27:02 INFO InMemoryFileIndex: It took 13 ms to list leaf files for 1 paths.
24/03/06 21:27:05 INFO FileSourceStrategy: Pushed Filters: 
24/03/06 21:27:05 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#0, None)) > 0)
24/03/06 21:27:06 INFO CodeGenerator: Code generated in 282.958205 ms
24/03/06 21:27:06 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 200.8 KiB, free 434.2 MiB)
24/03/06 21:27:06 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.7 KiB, free 434.2 MiB)
24/03/06 21:27:06 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on host-192-168-2-147-

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    NULL|       NULL|            CA|         200304.0|NULL|HOND|        PA|   GY|

In [3]:
dataframe.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [4]:
# Row count of CSV file
print(f"Number of rows: {dataframe.count()}")

24/03/06 21:27:43 INFO BlockManagerInfo: Removed broadcast_5_piece0 on 192.168.2.250:10006 in memory (size: 10.0 KiB, free: 366.2 MiB)
24/03/06 21:27:43 INFO BlockManagerInfo: Removed broadcast_5_piece0 on host-192-168-2-147-de1:10005 in memory (size: 10.0 KiB, free: 434.3 MiB)
24/03/06 21:27:43 INFO BlockManagerInfo: Removed broadcast_3_piece0 on host-192-168-2-147-de1:10005 in memory (size: 13.0 KiB, free: 434.3 MiB)
24/03/06 21:27:43 INFO BlockManagerInfo: Removed broadcast_3_piece0 on 192.168.2.250:10006 in memory (size: 13.0 KiB, free: 366.2 MiB)
24/03/06 21:27:43 INFO BlockManagerInfo: Removed broadcast_3_piece0 on 192.168.2.250:10005 in memory (size: 13.0 KiB, free: 366.3 MiB)
24/03/06 21:27:43 INFO BlockManagerInfo: Removed broadcast_4_piece0 on host-192-168-2-147-de1:10005 in memory (size: 34.8 KiB, free: 434.4 MiB)
24/03/06 21:27:43 INFO FileSourceStrategy: Pushed Filters: 
24/03/06 21:27:43 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/06 21:27:43 INFO BlockManagerInfo:

Number of rows: 13077724


24/03/06 21:27:46 INFO SparkContext: Starting job: count at NativeMethodAccessorImpl.java:0
24/03/06 21:27:46 INFO DAGScheduler: Got job 4 (count at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/06 21:27:46 INFO DAGScheduler: Final stage: ResultStage 5 (count at NativeMethodAccessorImpl.java:0)
24/03/06 21:27:46 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 4)
24/03/06 21:27:46 INFO DAGScheduler: Missing parents: List()
24/03/06 21:27:46 INFO DAGScheduler: Submitting ResultStage 5 (MapPartitionsRDD[20] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/06 21:27:46 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 12.5 KiB, free 434.1 MiB)
24/03/06 21:27:46 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 5.9 KiB, free 434.1 MiB)
24/03/06 21:27:46 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on host-192-168-2-147-de1:10005 (size: 5.9 KiB, free: 

In [5]:
# Number of partitions in the underlying RDD
print(f"Number of partitions: {dataframe.rdd.getNumPartitions()}")

Number of partitions: 16


24/03/06 21:28:03 INFO FileSourceStrategy: Pushed Filters: 
24/03/06 21:28:03 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/06 21:28:03 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 200.7 KiB, free 433.9 MiB)
24/03/06 21:28:03 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 433.9 MiB)
24/03/06 21:28:03 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on host-192-168-2-147-de1:10005 (size: 34.8 KiB, free: 434.3 MiB)
24/03/06 21:28:03 INFO SparkContext: Created broadcast 9 from javaToPython at NativeMethodAccessorImpl.java:0
24/03/06 21:28:03 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.


In [6]:
# Drop the columns VIN, Latitude, and Longitude
dataframe = dataframe.drop("VIN", "Latitude", "Longitude")

In [7]:
from pyspark.sql.functions import col

# Add new column from Fine amount as float
dataframe = dataframe.withColumn("FineAmountFloat", col("Fine amount").cast("float"))

# Show the DataFrame to verify the new column
dataframe.show()


24/03/06 21:29:22 INFO FileSourceStrategy: Pushed Filters: 
24/03/06 21:29:22 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/06 21:29:22 INFO BlockManagerInfo: Removed broadcast_8_piece0 on 192.168.2.250:10006 in memory (size: 5.9 KiB, free: 366.3 MiB)
24/03/06 21:29:22 INFO BlockManagerInfo: Removed broadcast_8_piece0 on host-192-168-2-147-de1:10005 in memory (size: 5.9 KiB, free: 434.3 MiB)
24/03/06 21:29:22 INFO BlockManagerInfo: Removed broadcast_7_piece0 on host-192-168-2-147-de1:10005 in memory (size: 9.2 KiB, free: 434.3 MiB)
24/03/06 21:29:22 INFO BlockManagerInfo: Removed broadcast_7_piece0 on 192.168.2.250:10006 in memory (size: 9.2 KiB, free: 366.3 MiB)
24/03/06 21:29:22 INFO BlockManagerInfo: Removed broadcast_7_piece0 on 192.168.2.250:10005 in memory (size: 9.2 KiB, free: 366.3 MiB)
24/03/06 21:29:22 INFO CodeGenerator: Code generated in 52.762847 ms
24/03/06 21:29:22 INFO MemoryStore: Block broadcast_10 stored as values in memory (estimated size 200.7 KiB, free 433.7 

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+---------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|Agency Description|Color Description|Body Style Description|FineAmountFloat|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+---------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    NULL|       NULL|            CA|         200304.0|HOND|        PA|   GY|     13147 WELBY WAY|01521|   1.

24/03/06 21:29:22 INFO TaskSetManager: Finished task 0.0 in stage 6.0 (TID 35) in 184 ms on 192.168.2.250 (executor 0) (1/1)
24/03/06 21:29:22 INFO DAGScheduler: ResultStage 6 (showString at NativeMethodAccessorImpl.java:0) finished in 0.216 s
24/03/06 21:29:22 INFO DAGScheduler: Job 5 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/06 21:29:22 INFO TaskSchedulerImpl: Removed TaskSet 6.0, whose tasks have all completed, from pool 
24/03/06 21:29:22 INFO TaskSchedulerImpl: Killing all running tasks in stage 6: Stage finished
24/03/06 21:29:22 INFO DAGScheduler: Job 5 finished: showString at NativeMethodAccessorImpl.java:0, took 0.225186 s
24/03/06 21:29:22 INFO CodeGenerator: Code generated in 67.193796 ms


In [8]:
# Check that the new column was added
dataframe.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)
 |-- FineAmountFloat: float (nullable = true)



In [9]:
# Find the maximum fine amount
max_fine = dataframe.agg({"FineAmountFloat": "max"}).collect()[0][0]

# Print the maximum fine amount
print(f"Maximum fine amount: {max_fine}")

24/03/06 21:30:29 INFO FileSourceStrategy: Pushed Filters: 
24/03/06 21:30:29 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/06 21:30:29 INFO CodeGenerator: Code generated in 36.4174 ms
24/03/06 21:30:29 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 200.7 KiB, free 433.7 MiB)
24/03/06 21:30:29 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 433.7 MiB)
24/03/06 21:30:29 INFO BlockManagerInfo: Added broadcast_12_piece0 in memory on host-192-168-2-147-de1:10005 (size: 34.8 KiB, free: 434.3 MiB)
24/03/06 21:30:29 INFO SparkContext: Created broadcast 12 from collect at /tmp/ipykernel_19861/741873320.py:2
24/03/06 21:30:29 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/03/06 21:30:29 INFO DAGScheduler: Registering RDD 33 (collect at /tmp/ipykernel_19861/741873320.py:2) as input to shuffle 1
24/03/06 21:30:29 INFO

Maximum fine amount: 1100.0


24/03/06 21:30:37 INFO DAGScheduler: Final stage: ResultStage 9 (collect at /tmp/ipykernel_19861/741873320.py:2)
24/03/06 21:30:37 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 8)
24/03/06 21:30:37 INFO DAGScheduler: Missing parents: List()
24/03/06 21:30:37 INFO DAGScheduler: Submitting ResultStage 9 (MapPartitionsRDD[36] at collect at /tmp/ipykernel_19861/741873320.py:2), which has no missing parents
24/03/06 21:30:37 INFO MemoryStore: Block broadcast_14 stored as values in memory (estimated size 13.2 KiB, free 433.6 MiB)
24/03/06 21:30:37 INFO MemoryStore: Block broadcast_14_piece0 stored as bytes in memory (estimated size 6.2 KiB, free 433.6 MiB)
24/03/06 21:30:37 INFO BlockManagerInfo: Added broadcast_14_piece0 in memory on host-192-168-2-147-de1:10005 (size: 6.2 KiB, free: 434.3 MiB)
24/03/06 21:30:37 INFO SparkContext: Created broadcast 14 from broadcast at DAGScheduler.scala:1580
24/03/06 21:30:37 INFO DAGScheduler: Submitting 1 missing tasks from ResultStage 

In [10]:
# Count how many fines have this maximum amount
max_fine_count = dataframe.filter(col("FineAmountFloat") == max_fine).count()

print(f"Count of maximum fines: {max_fine_count}")

24/03/06 21:31:06 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Fine amount)
24/03/06 21:31:06 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Fine amount#33),(cast(Fine amount#33 as float) = 1100.0)
24/03/06 21:31:06 INFO CodeGenerator: Code generated in 21.019904 ms
24/03/06 21:31:06 INFO MemoryStore: Block broadcast_15 stored as values in memory (estimated size 200.7 KiB, free 433.4 MiB)
24/03/06 21:31:06 INFO MemoryStore: Block broadcast_15_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 433.4 MiB)
24/03/06 21:31:06 INFO BlockManagerInfo: Added broadcast_15_piece0 in memory on host-192-168-2-147-de1:10005 (size: 34.8 KiB, free: 434.2 MiB)
24/03/06 21:31:06 INFO SparkContext: Created broadcast 15 from count at NativeMethodAccessorImpl.java:0
24/03/06 21:31:06 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/03/06 21:31:06 INFO DAGScheduler: Registering RDD 40 (count at

Count of maximum fines: 626


24/03/06 21:31:13 INFO SparkContext: Starting job: count at NativeMethodAccessorImpl.java:0
24/03/06 21:31:13 INFO DAGScheduler: Got job 9 (count at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/06 21:31:13 INFO DAGScheduler: Final stage: ResultStage 12 (count at NativeMethodAccessorImpl.java:0)
24/03/06 21:31:13 INFO DAGScheduler: Parents of final stage: List(ShuffleMapStage 11)
24/03/06 21:31:13 INFO DAGScheduler: Missing parents: List()
24/03/06 21:31:13 INFO DAGScheduler: Submitting ResultStage 12 (MapPartitionsRDD[43] at count at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/06 21:31:13 INFO MemoryStore: Block broadcast_17 stored as values in memory (estimated size 12.5 KiB, free 433.4 MiB)
24/03/06 21:31:13 INFO MemoryStore: Block broadcast_17_piece0 stored as bytes in memory (estimated size 5.9 KiB, free 433.4 MiB)
24/03/06 21:31:13 INFO BlockManagerInfo: Added broadcast_17_piece0 in memory on host-192-168-2-147-de1:10005 (size: 5.9 KiB, 

In [14]:
from pyspark.sql import functions as pyfuncs

# Show the top 20 most frequent vehicle makes, and their frequencies
vehicle_makes = dataframe.groupBy("Make").count()

sorted_makes = vehicle_makes.orderBy("Count", ascending=False).show(20)

24/03/06 21:34:55 INFO FileSourceStrategy: Pushed Filters: 
24/03/06 21:34:55 INFO FileSourceStrategy: Post-Scan Filters: 
24/03/06 21:34:55 INFO MemoryStore: Block broadcast_27 stored as values in memory (estimated size 200.7 KiB, free 433.4 MiB)
24/03/06 21:34:55 INFO MemoryStore: Block broadcast_27_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 433.4 MiB)
24/03/06 21:34:55 INFO BlockManagerInfo: Added broadcast_27_piece0 in memory on host-192-168-2-147-de1:10005 (size: 34.8 KiB, free: 434.2 MiB)
24/03/06 21:34:55 INFO SparkContext: Created broadcast 27 from showString at NativeMethodAccessorImpl.java:0
24/03/06 21:34:55 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/03/06 21:34:55 INFO DAGScheduler: Registering RDD 71 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 6
24/03/06 21:34:55 INFO DAGScheduler: Got map stage job 16 (showString at NativeMethodAccess

+----+-------+
|Make|  count|
+----+-------+
|TOYT|2150768|
|HOND|1479996|
|FORD|1116235|
|NISS| 945133|
|CHEV| 892676|
| BMW| 603092|
|MERZ| 543298|
|VOLK| 432030|
|HYUN| 404917|
|DODG| 391686|
|LEXS| 368420|
| KIA| 328155|
|JEEP| 316300|
|AUDI| 255395|
|MAZD| 242344|
|OTHR| 205546|
| GMC| 184889|
|INFI| 174315|
|CHRY| 159948|
|SUBA| 154640|
+----+-------+
only showing top 20 rows



24/03/06 21:35:02 INFO TaskSetManager: Finished task 13.0 in stage 22.0 (TID 134) in 1729 ms on 192.168.2.250 (executor 1) (16/16)
24/03/06 21:35:02 INFO TaskSchedulerImpl: Removed TaskSet 22.0, whose tasks have all completed, from pool 
24/03/06 21:35:02 INFO DAGScheduler: ShuffleMapStage 22 (showString at NativeMethodAccessorImpl.java:0) finished in 7.273 s
24/03/06 21:35:02 INFO DAGScheduler: looking for newly runnable stages
24/03/06 21:35:02 INFO DAGScheduler: running: Set()
24/03/06 21:35:02 INFO DAGScheduler: waiting: Set()
24/03/06 21:35:02 INFO DAGScheduler: failed: Set()
24/03/06 21:35:02 INFO ShufflePartitionsUtil: For shuffle(6), advisory target size: 67108864, actual target size 1048576, minimum partition size: 1048576
24/03/06 21:35:02 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
24/03/06 21:35:02 INFO SparkContext: Starting job: showString at NativeM

In [18]:
# Expand abbreviations in the color column using a User Defined Function
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a dictionary mapping color abbreviations to their full names
COLORS = {
    'AL': 'Aluminum', 'AM': 'Amber', 'BG': 'Beige', 'BK': 'Black',
    'BL': 'Blue', 'BN': 'Brown', 'BR': 'Brown', 'BZ': 'Bronze',
    'CH': 'Charcoal', 'DK': 'Dark', 'GD': 'Gold', 'GO': 'Gold',
    'GN': 'Green', 'GY': 'Gray', 'GT': 'Granite', 'IV': 'Ivory',
    'LT': 'Light', 'OL': 'Olive', 'OR': 'Orange', 'MR': 'Maroon',
    'PK': 'Pink', 'RD': 'Red', 'RE': 'Red', 'SI': 'Silver', 'SL': 'Silver',
    'SM': 'Smoke', 'TN': 'Tan', 'VT': 'Violet', 'WT': 'White', 'WH': 'White',
    'YL': 'Yellow', 'YE': 'Yellow', 'UN': 'Unknown'
}

# The function takes a color abbreviation and returns the full color name using the dictionary defined above
# if there is no key matching the original color, the original color is returned
def expand_color_abbreviation(color_abbr):
    return COLORS.get(color_abbr, color_abbr)

# chekc the udf with Spark, specifying the return type as StringType
expand_color_udf = udf(expand_color_abbreviation, StringType())

# apply the udf to the Color column of the DataFrame, creates a new column ColorLong with expanded color names
dataframe = dataframe.withColumn("ColorLong", expand_color_udf(dataframe["Color"]))

In [19]:
# Find the most frequent color value for Toyotas (TOYT)
dataframe.filter(dataframe["Make"] == "TOYT").groupBy("ColorLong").count().orderBy("count", ascending=False).show(1)

24/03/06 21:43:08 INFO FileSourceStrategy: Pushed Filters: IsNotNull(Make),EqualTo(Make,TOYT)
24/03/06 21:43:08 INFO FileSourceStrategy: Post-Scan Filters: isnotnull(Make#25),(Make#25 = TOYT)
24/03/06 21:43:09 INFO MemoryStore: Block broadcast_33 stored as values in memory (estimated size 200.7 KiB, free 432.7 MiB)
24/03/06 21:43:09 INFO MemoryStore: Block broadcast_33_piece0 stored as bytes in memory (estimated size 34.8 KiB, free 432.6 MiB)
24/03/06 21:43:09 INFO BlockManagerInfo: Added broadcast_33_piece0 in memory on host-192-168-2-147-de1:10005 (size: 34.8 KiB, free: 434.1 MiB)
24/03/06 21:43:09 INFO SparkContext: Created broadcast 33 from showString at NativeMethodAccessorImpl.java:0
24/03/06 21:43:09 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/03/06 21:43:09 INFO DAGScheduler: Registering RDD 93 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 8
24/03/06 21:43:09 INFO

+---------+------+
|ColorLong| count|
+---------+------+
|     Gray|489697|
+---------+------+
only showing top 1 row



24/03/06 21:43:16 INFO TaskSetManager: Finished task 14.0 in stage 28.0 (TID 169) in 1788 ms on 192.168.2.250 (executor 1) (16/16)
24/03/06 21:43:16 INFO TaskSchedulerImpl: Removed TaskSet 28.0, whose tasks have all completed, from pool 
24/03/06 21:43:16 INFO DAGScheduler: ShuffleMapStage 28 (showString at NativeMethodAccessorImpl.java:0) finished in 7.638 s
24/03/06 21:43:16 INFO DAGScheduler: looking for newly runnable stages
24/03/06 21:43:16 INFO DAGScheduler: running: Set()
24/03/06 21:43:16 INFO DAGScheduler: waiting: Set()
24/03/06 21:43:16 INFO DAGScheduler: failed: Set()
24/03/06 21:43:16 INFO ShufflePartitionsUtil: For shuffle(8), advisory target size: 67108864, actual target size 1048576, minimum partition size: 1048576
24/03/06 21:43:16 INFO HashAggregateExec: spark.sql.codegen.aggregate.map.twolevel.enabled is set to true, but current version of codegened fast hashmap does not support this aggregate.
24/03/06 21:43:16 INFO SparkContext: Starting job: showString at NativeM